# Tutorial 4: Type Safety

*Well-Typed Programs Don't Go Wrong*

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/types-typed-passages/blob/main/notebooks/04-type-safety.ipynb)

---

## A Note from the Archives

> *Year 825, Capital Archives*
>
> "Two theorems," Brennis announced to the assembled scholars. "Preservation and Progress. Together, they guarantee that well-typed passages never 'go wrong.'"
>
> "What do you mean by 'go wrong'?" asked Elder Kellum.
>
> "A passage 'goes wrong' when it gets stuck — when it's not a value, but cannot reduce further. An ill-typed application, like trying to apply a boolean to a natural. In untyped calculus, such expressions can arise during reduction. In the typed calculus, they cannot."
>
> "Impossible?"
>
> "Provably impossible."

---

## Learning Objectives

By the end of this tutorial, you will be able to:

1. Define what it means for a program to "go wrong"
2. State the Preservation theorem
3. State the Progress theorem
4. Explain how Preservation + Progress = Type Safety
5. Trace type preservation through reduction steps

## Setup

In [ ]:
import pandas as pd

BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/densworld-datasets/main/data/"

expressions_df = pd.read_csv(BASE_URL + "typed_passage_expressions.csv")
traces_df = pd.read_csv(BASE_URL + "normalization_traces.csv")

print(f"Loaded {len(traces_df)} reduction steps")

## 1. What Does "Go Wrong" Mean?

A program "goes wrong" when it gets **stuck**:
- It's not a value (not a lambda abstraction)
- It cannot reduce further

Example of getting stuck (in untyped calculus):
- $(\text{TRUE}\ 5)$ — applying a boolean to a number
- TRUE = $\lambda x.\lambda y.x$, so $(\text{TRUE}\ 5) = (\lambda x.\lambda y.x)\ 5 \to \lambda y.5$
- But what if we accidentally wrote $(5\ \text{TRUE})$? Numbers aren't functions!

In typed calculus, $(5\ \text{TRUE})$ would be rejected by the type system.

In [ ]:
# Values vs non-values in our traces
print("What are values in simply typed lambda calculus?")
print()
print("  Values are lambda abstractions: λx.M")
print("  Non-values are applications: (M N)")
print()

# Count values vs non-values in trace endpoints
endpoints = traces_df[traces_df['is_value'] == True]
print(f"  Trace endpoints that are values: {len(endpoints)}")
print(f"  (All typed reductions end in values)")

## 2. The Preservation Theorem

**Preservation** (also called Subject Reduction):

> If $\Gamma \vdash M : \tau$ and $M \to M'$, then $\Gamma \vdash M' : \tau$

In words: **Reduction preserves types.** If an expression has type $\tau$, then after one reduction step, it still has type $\tau$.

In [ ]:
# Show preservation in action
print("Preservation Example:")
print()
print("  (λx.x)(λy.y) : τ→τ")
print("  →")
print("  λy.y : τ→τ")
print()
print("  The type τ→τ is preserved after reduction!")

In [ ]:
# Trace a reduction showing type preservation
trace_1 = traces_df[traces_df['trace_id'] == 'NT-001']

print("Reduction trace showing preservation:")
print(f"Expression type: {trace_1.iloc[0]['type']}")
print()
for _, row in trace_1.iterrows():
    value_marker = " [VALUE]" if row['is_value'] else ""
    print(f"  Step {row['step_number']}: {row['current_term']}{value_marker}")

print(f"\nType remains {trace_1.iloc[0]['type']} throughout!")

## 3. The Progress Theorem

**Progress**:

> If $\emptyset \vdash M : \tau$ (M is well-typed in the empty context), then either:
> - $M$ is a value, or
> - There exists $M'$ such that $M \to M'$

In words: **Well-typed terms don't get stuck.** They either are already values, or can take another step.

In [ ]:
# Show progress in action
print("Progress Example:")
print()
print("  Consider (λf.λx.(f x))(λy.y)")
print("  Type: τ→τ")
print()
print("  Is it a value? No (it's an application)")
print("  Can it reduce? Yes!")
print("  → λx.((λy.y) x)")
print("  → λx.x")
print()
print("  Now λx.x is a value. Progress satisfied!")

In [ ]:
# Show a multi-step trace demonstrating progress
trace_3 = traces_df[traces_df['trace_id'] == 'NT-003']

print("Progress through a reduction:")
print()
for _, row in trace_3.iterrows():
    if row['is_value']:
        status = "VALUE (done)"
    else:
        status = "can reduce"
    print(f"  Step {row['step_number']}: {status}")
    print(f"    {row['current_term'][:50]}..." if len(str(row['current_term'])) > 50 else f"    {row['current_term']}")

## 4. Type Safety = Preservation + Progress

Together, these theorems give us **Type Safety**:

> Well-typed programs don't go wrong.

Why? By induction on the reduction sequence:

1. **Base case**: $M_0$ is well-typed (by assumption)
2. **Inductive step**: If $M_n$ is well-typed:
   - By **Progress**: $M_n$ is a value OR $M_n \to M_{n+1}$
   - By **Preservation**: If $M_n \to M_{n+1}$, then $M_{n+1}$ is well-typed
3. **Conclusion**: Every step in the reduction is well-typed. We never get stuck.

In [ ]:
# Visualize type safety
print("Type Safety Visualization:")
print()
print("  M₀ : τ  (well-typed)")
print("   ↓ Progress: can reduce")
print("  M₁ : τ  (Preservation: still type τ)")
print("   ↓ Progress: can reduce")
print("  M₂ : τ  (Preservation: still type τ)")
print("   ↓")
print("  ...")
print("   ↓ Progress: is a value!")
print("  Mₙ : τ  [VALUE]")
print()
print("  At no point can we 'go wrong' (get stuck).")

## 5. Tracing Preservation Through Reductions

Let's trace a more complex example: adding Church numerals.

In [ ]:
# PLUS ONE TWO reduction
trace_5 = traces_df[traces_df['trace_id'] == 'NT-005']

print("Tracing PLUS ONE TWO:")
print(f"Type: {trace_5.iloc[0]['type']}")
print()

for _, row in trace_5.iterrows():
    term = row['current_term']
    if len(term) > 60:
        term = term[:57] + "..."
    status = " [VALUE]" if row['is_value'] else ""
    print(f"  Step {row['step_number']}: {term}{status}")

print()
print("  Final value: f (f (f x)) = THREE")
print("  Type preserved as τ throughout!")

## 6. What Progress Excludes

Progress only holds for **closed** terms (empty context).

With free variables, an expression like $x\ y$ is well-typed in context $\{x:\tau_1\to\tau_2, y:\tau_1\}$, but cannot reduce (we don't know what $x$ is).

This is fine — free variables represent "holes" waiting to be filled.

In [ ]:
# Open vs closed terms
print("Open vs Closed Terms:")
print()
print("  CLOSED (no free variables):")
print("    λx.x                    - can reduce to value")
print("    (λx.x)(λy.y)           - can reduce")
print()
print("  OPEN (free variables):")
print("    x                       - stuck (but that's OK)")
print("    x y                     - stuck (that's OK)")
print("    λx.(x y)               - value! (y is free but under λ)")
print()
print("  Progress applies only to closed terms.")

## 7. Why This Matters for the Archive

The Year 800 reduction engine halt happened because an untyped expression got into an infinite loop. Type Safety guarantees two things:

1. **No runtime type errors**: We won't accidentally apply a boolean to a natural
2. **No stuck states**: If computation is possible, it will proceed

But note: Type Safety does **not** guarantee termination! That requires the next theorem (Strong Normalization).

In [ ]:
# Type safety vs termination
print("What Type Safety Guarantees:")
print()
print("  ✓ No type errors during reduction")
print("  ✓ No stuck states (except values)")
print("  ✓ If you can reduce, reduction is safe")
print()
print("What Type Safety Does NOT Guarantee:")
print()
print("  ✗ Termination (reduction might go forever)")
print("    ... but that requires Omega, which is untypable!")
print("    ... so in STLC, type safety + termination = Strong Normalization")

## Summary

In this tutorial, we learned:

1. A program "goes wrong" when it gets **stuck** (not a value, can't reduce)
2. **Preservation**: Reduction preserves types ($M : \tau$ and $M \to M'$ implies $M' : \tau$)
3. **Progress**: Well-typed closed terms are values or can reduce
4. **Type Safety** = Preservation + Progress: well-typed programs don't go wrong
5. This applies to **closed** terms (empty context)

In the next tutorial, we'll see that types guarantee something even stronger: **termination**.

---

## Exercises

1. Trace the type through the reduction: $(\lambda x.\lambda y.x)\ a\ b \to^* a$

2. Why is $(5\ \text{TRUE})$ not well-typed? Which typing rule fails?

3. Give an example of an open term that is well-typed but stuck.

4. If we had a term that reduced forever, would it violate Type Safety? Why or why not?

---

*Next: Tutorial 5 - Strong Normalization*